In [4]:
!pip install -q transformers accelerate bitsandbytes rouge-score

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoawq 0.2.7.post3 requires datasets>=2.20, but you have datasets 2.19.1 which is incompatible.
axolotl 0.9.2 requires datasets==3.5.1, but you have datasets 2.19.1 which is incompatible.
axolotl 0.9.2 requires evaluate==0.4.1, but you have evaluate 0.4.3 which is incompatible.
axolotl 0.9.2 requires hf_xet==1.1.0, but you have hf-xet 1.1.3 which is incompatible.
axolotl 0.9.2 requires huggingface_hub==0.31.0, but you have huggingface-hub 0.33.0 which is incompatible.
trl 0.17.0 requires datasets>=3.0.0, but you have datasets 2.19.1 which is incompatible.


In [11]:
pip uninstall -y evaluate datasets huggingface_hub axolotl

Found existing installation: evaluate 0.4.3
Uninstalling evaluate-0.4.3:
  Successfully uninstalled evaluate-0.4.3
Found existing installation: datasets 3.5.1
Uninstalling datasets-3.5.1:
  Successfully uninstalled datasets-3.5.1
Found existing installation: huggingface-hub 0.31.0
Uninstalling huggingface-hub-0.31.0:
  Successfully uninstalled huggingface-hub-0.31.0
Found existing installation: axolotl 0.9.2
Uninstalling axolotl-0.9.2:
  Successfully uninstalled axolotl-0.9.2
Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install -q "huggingface_hub==0.33.0" "datasets==3.5.1" "evaluate==0.4.3"

In [9]:
!pip install bert_score

In [14]:
!sudo apt-get install -y libaio1 libstdc++6 build-essential

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libaio1 is already the newest version (0.3.112-5).
build-essential is already the newest version (12.8ubuntu1.1).
libstdc++6 is already the newest version (10.5.0-1ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [2]:
import os
import json
import random
import torch
import evaluate
import pandas as pd
from tqdm import tqdm
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [3]:
notebook_login()

In [7]:
def load_local_dataset(filepath):
    with open(filepath, 'r') as f:
        data = [json.loads(line) for line in f]
    return data

In [7]:
import json
import evaluate

def evaluate_predictions_from_file(pred_file):
    with open(pred_file, 'r') as f:
        data = [json.loads(line) for line in f]

    preds = [d["model_output"] for d in data]
    refs = [d["expected_output"] for d in data]

    # ROUGE
    rouge = evaluate.load("rouge")
    rouge_score = rouge.compute(predictions=preds, references=refs, use_stemmer=True)

    # BLEU
    bleu = evaluate.load("bleu")
    bleu_score = bleu.compute(predictions=preds, references=[[ref] for ref in refs])

    # METEOR
    meteor = evaluate.load("meteor")
    meteor_score = meteor.compute(predictions=preds, references=refs)

    # ChrF
    chrf = evaluate.load("chrf")
    chrf_score = chrf.compute(predictions=preds, references=refs)

    # BERTScore
    bertscore = evaluate.load("bertscore")
    bert_score = bertscore.compute(predictions=preds, references=refs, lang="en")

    return {
        "rougeL": round(rouge_score["rougeL"] * 100, 2),
        "bleu": round(bleu_score["bleu"] * 100, 2),
        "meteor": round(meteor_score["meteor"] * 100, 2),
        "chrf": round(chrf_score["score"], 2),
        "bertscore_f1": round(sum(bert_score["f1"]) / len(bert_score["f1"]) * 100, 2)
    }

In [9]:
def run_model_inference(model_name_or_path, test_data):
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        device_map="auto",
        load_in_8bit=True
    )
    model.eval()

    predictions = []
    for item in tqdm(test_data):
        prompt = item["prompt"]
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.inference_mode():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=False
            )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if generated_text.startswith(prompt):
            generated_text = generated_text[len(prompt):].strip()
        predictions.append(generated_text)

    del model
    torch.cuda.empty_cache()

    return predictions

In [13]:
def run_single_model_evaluation(model_id, model_path, test_data, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    refs = [item["expected_output"] for item in test_data]

    print(f"Running inference for: {model_id}")
    preds = run_model_inference(model_path, test_data)

    # Save outputs
    output_file = f"{output_dir}/{model_id}_outputs.jsonl"
    with open(output_file, "w") as f:
        for item, pred in zip(test_data, preds):
            f.write(json.dumps({
                "prompt": item["prompt"],
                "expected_output": item["expected_output"],
                "model_output": pred
            }) + "\n")

    print(f"{model_id} inference completed. Output saved to {output_file}")

In [10]:
def sample_dataset(dataset, k):
    return random.sample(dataset, k)

In [11]:
test_data = load_local_dataset("validation.jsonl")
sampled_data = sample_dataset(test_data, 100)

In [14]:
run_single_model_evaluation("hermes", "capstone-research/Hermes-2-Pro-Mistral-7B-Travel-ChatBot", sampled_data, "outputs")

Running inference for: hermes


tokenizer_config.json:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

100%|██████████| 100/100 [15:45<00:00,  9.46s/it]

hermes inference completed. Output saved to outputs/hermes_outputs.jsonl


In [13]:
run_single_model_evaluation("openchat", "capstone-research/OpenChat-3.5-Travel-ChatBot", sampled_data, "outputs")

Running inference for: openchat


tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/148 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 100/100 [45:06<00:00, 27.06s/it]

openchat inference completed. Output saved to outputs/openchat_outputs.jsonl


In [15]:
run_single_model_evaluation("tinyllama", "capstone-research/TinyLlama-1.1B-Travel-ChatBot", sampled_data, "outputs")

Running inference for: tinyllama


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

...

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

100%|██████████| 100/100 [20:19<00:00, 12.20s/it]

tinyllama inference completed. Output saved to outputs/tinyllama_outputs.jsonl


In [5]:
def display_final_summary(output_dir):
    print("\nFinal Evaluation Summary:\n")
    summary = []
    for file in os.listdir(output_dir):
        if file.endswith("_outputs.jsonl"):
            model_id = file.replace("_outputs.jsonl", "")
            score = evaluate_predictions_from_file(os.path.join(output_dir, file))
            summary.append({"Model": model_id, **score})

    df = pd.DataFrame(summary).sort_values(by="bertscore_f1", ascending=False).reset_index(drop=True)
    print(df.to_markdown(index=False))
    return df

In [39]:
!rm -rf ~/.cache/huggingface/modules/evaluate_modules

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
display_final_summary("outputs")


Final Evaluation Summary:



[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

| Model     |   rougeL |   bleu |   meteor |   chrf |   bertscore_f1 |
|:----------|---------:|-------:|---------:|-------:|---------------:|
| tinyllama |    31.83 |  19.61 |    52.75 |  53.74 |          89.53 |
| openchat  |    29.72 |  17.88 |    49.18 |  51    |          88.42 |
| hermes    |    42.37 |  37.8  |    47.78 |  48.91 |          74.27 |


,Model,rougeL,bleu,meteor,chrf,bertscore_f1
0,tinyllama,31.83,19.61,52.75,53.74,89.53
1,openchat,29.72,17.88,49.18,51.00,88.42
2,hermes,42.37,37.80,47.78,48.91,74.27
